In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Path to your saved model directory
model_dir = './roberta_sentiment_model_final/'

# Load the model
model = RobertaForSequenceClassification.from_pretrained(model_dir)

# Load the tokenizer (if you want to preprocess new text)
tokenizer = RobertaTokenizer.from_pretrained(model_dir)